In [1]:
from ddrnet_23_slim import DualResNet_imagenet, DualResNet
import numpy as np
import keras
import tensorflow as tf
import sys
from Utility import *
from Dataset_helper import *
import tf2onnx 
import onnx
import onnxruntime as rt 
import time
from threading import Thread
from queue import Queue

import time
import cv2
from datetime import timedelta
from timeit import default_timer as timer
from imutils.video import FileVideoStream
from imutils.video import FPS
import imutils

from evaulate_model import Evaluate
import pandas as pd
import focal_loss

In [2]:
model_name = "onnx_final/ddrnet336_600.onnx"

In [3]:
# load onnx model

providers = ['CPUExecutionProvider']
sess_options = rt.SessionOptions()

sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL


sess = rt.InferenceSession(model_name, sess_options=sess_options,
                           providers=providers) 


output_names = [n.name for n in onnx.load(model_name).graph.output]

In [4]:
# onnx model for evaulate object
class Onnx_model:
    def __init__(self, model=None, names=None):
        self.sess = model
        self.names = names
        
    def get_predict(self, frame):
        if frame is not None:
            pred_img = self.sess.run(self.names, {'input': frame})[0]
            return np.uint8(((np.argmax(pred_img[0,:,:,:], axis=-1)/12)+0)*255)

In [5]:
# tensorflow model and tf model func for evaluate object
class UpdatedMeanIoU(tf.keras.metrics.MeanIoU):
    def __init__(self,
               y_true=None,
               y_pred=None,
               num_classes=None,
               name=None,
               dtype=None):
        super(UpdatedMeanIoU, self).__init__(num_classes = num_classes,name=name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.math.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)
    
    def get_config(self):
        config = super().get_config().copy()
        return config


tf_model = tf.keras.models.load_model('saved_model_final/ddrnet336_600_tf', 
                                      custom_objects={'UpdatedMeanIoU': UpdatedMeanIoU})


class TF:
    def __init__(self, model=None, names=None):
        self.model = model
        self.names = names
        
    def get_predict(self, frame):
        if frame is not None:
            with tf.device('/GPU:0'):
                pred_img = self.model.predict(frame)
            return np.uint8(((np.argmax(pred_img[0,:,:,:], axis=-1)/12)+0)*255)

In [5]:
def transform_func(img=None):
    if img is not None:
        
        frame = cv2.resize(img, (600, 336), cv2.INTER_NEAREST)/255
        frame = np.expand_dims(frame, 0).astype('float32')
        
        
        return frame
    else:
        return None

In [6]:
# save CPU, RAM, GPU usage as csv
def make_dataframe(gpu, cpu, memory, t1, t2, t1_m, t2_m):
      
    time = [[t1[i] ,t2[i], t1_m[i], t2_m[i]]  for i in range(len(eval_his.t1)-1)]
    time = pd.DataFrame(time, columns=['t1', 't2', 't1_model', 't2_model'])
    cpu = pd.DataFrame(cpu, columns=['cpu0', 'cpu1', 'cpu2', 'cpu3', 'cpu4', 'cpu5', 'cpu6', 'cpu7'])
    gpu = pd.DataFrame(gpu)
    memory = pd.DataFrame(memory, columns=['total', 'available', 'percent', 'used', 'free'])
    
    return pd.concat([gpu, cpu, memory, time], axis=1)  

In [9]:
size = ['336_600', '400_720','720_1280','1080_1920']

In [10]:
# test every model on cpu/cuda for every resolutin (size)

for k in size:
    
    model_name = f"onnx_final/ddrnet{k}.onnx"
    sess = rt.InferenceSession(model_name, sess_options=sess_options,
                           providers=providers) 
    onnx_model = Onnx_model(model=sess, names=output_names).get_predict

#     tf_model = tf.keras.models.load_model(f'saved_model_final/ddrnet{k}_tf', 
#                                       custom_objects={'UpdatedMeanIoU': UpdatedMeanIoU})



#     tff = TF(model=tf_model).get_predict
    
    x1 = int(k.split('_')[0])
    x2 = int(k.split('_')[1])
    
    
    def transform_func(img=None):
        if img is not None:

            frame = cv2.resize(img, (x2, x1), cv2.INTER_NEAREST)/255
            frame = np.expand_dims(frame, 0).astype('float32')

            return frame
        else:
            return None
    

    for i in range(1, 2):
        eval_his = Evaluate('IMG_7677k.mp4', 
                        onnx_model, transform_func, meas=True, max_num_f=400)

        eval_his.start_evaluate()

        df = make_dataframe(eval_his.gpu, eval_his.cpu, eval_his.memory, eval_his.t1, eval_his.t2, 
                   eval_his.t1_model, eval_his.t2_model)

        df.to_csv(f'test_obm_vyk_tf/gpu/1050ti/{k}/his{i}.csv', index=False)

[INFO] Press "q" to release video
[INFO] Starting video file read thread...
[INFO] Starting video file show thread...


KeyboardInterrupt: 